## Mapas Coropléticos

Um [mapa coroplético](https://en.wikipedia.org/wiki/Choropleth_map) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas de símbolos dimensionados são frequentemente mais precisos para ler, pois as pessoas tendem a ser melhores em estimar as diferenças proporcionais entre a área dos círculos do que entre os tons de cor. No entanto, mapas choropléticos são populares na prática e particularmente úteis quando muitos símbolos se tornam visualmente sobrecarregados.

Por exemplo, enquanto os Estados Unidos têm apenas 50 estados, eles têm milhares de condados dentro desses estados. Vamos construir um mapa choroplético da taxa de desemprego por condado, no ano de recessão de 2008. Em alguns casos, arquivos de entrada em GeoJSON ou TopoJSON podem incluir dados estatísticos que podemos visualizar diretamente. No entanto, neste caso, temos dois arquivos: nosso arquivo TopoJSON que inclui as características dos limites dos condados (`usa`), e um arquivo de texto separado que contém as estatísticas de desemprego:


In [ ]:
unemp = data.unemployment.url
unemp

Para integrar nossas fontes de dados, precisaremos novamente usar a transformação `lookup`, complementando nossos dados `geoshape` baseados em TopoJSON com as taxas de desemprego. Podemos então criar um mapa que inclui uma codificação de `color` para o campo `rate` obtido pela busca.


In [ ]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nas regiões das [Great Plains](https://en.wikipedia.org/wiki/Great_Plains) e nos estados das Montanhas exibem tanto taxas baixas **quanto** altas. Será que essa variação é significativa, ou é possivelmente um [artifício de amostras menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o domínio superior da escala (por exemplo, para `0.2`) para ajustar o mapeamento das cores.*

Uma preocupação central para mapas choropléticos é a escolha das cores. Acima, usamos o esquema padrão `'yellowgreenblue'` do Altair para mapas de calor. Abaixo, comparamos outros esquemas, incluindo um esquema _sequencial de uma única tonalidade_ (`teals`) que varia apenas na luminância, um esquema _sequencial multicolorido_ (`viridis`) que varia tanto em luminância quanto em tonalidade, e um esquema _divergente_ (`blueorange`) que usa um ponto médio branco:


In [ ]:
# utility function to generate a map specification for a provided color scheme
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

_Qual esquema de cor você acha que é mais eficaz? Por que? Modifique os mapas acima e use outros esquemas disponíveis, como descrito na [documentação dos Esquemas de Cor do Vega](https://vega.github.io/vega/docs/schemes/)._

## Projeções Cartográficas

Agora que já temos alguma experiência criando mapas, vamos dar uma olhada mais de perto nas projeções cartográficas. Como explicado pela [Wikipedia](https://en.wikipedia.org/wiki/Map_projection),

> _Todas as projeções de mapas necessariamente distorcem a superfície de alguma forma. Dependendo do objetivo do mapa, algumas distorções são aceitáveis e outras não; portanto, diferentes projeções de mapas existem para preservar algumas propriedades do corpo esférico à custa de outras propriedades._

Algumas das propriedades que podemos considerar incluem:

- _Área_: A projeção distorce os tamanhos das regiões?
- _Rumo_: Uma linha reta corresponde a uma direção constante de viagem?
- _Distância_: Linhas de comprimento igual correspondem a distâncias iguais na esfera?
- _Forma_: A projeção preserva as relações espaciais (ângulos) entre pontos?

Selecionar uma projeção apropriada depende, portanto, do caso de uso do mapa. Por exemplo, se estamos avaliando o uso da terra e a extensão da terra é importante, poderíamos escolher uma projeção que preserve a área. Se quisermos visualizar ondas de choque emanando de um terremoto, podemos focar o mapa no epicentro do terremoto e preservar as distâncias a partir daquele ponto. Ou, se quisermos auxiliar na navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções em termos da _superfície de projeção_. Projeções cilíndricas, por exemplo, projetam os pontos da superfície da esfera em torno de um cilindro; o cilindro "desenrolado" então fornece nosso mapa. Como descrevemos abaixo, podemos projetar alternativamente sobre a superfície de um cone (projeções cônicas) ou diretamente sobre um plano plano (projeções azimutais).

*Vamos primeiro construir nossa intuição interagindo com uma variedade de projeções! **[Abra o notebook online de Projeções Cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections).** Use os controles naquela página para selecionar uma projeção e explorar parâmetros de projeção, como o `scale` (zoom) e a tradução x/y (movimento). Os controles de rotação ([yaw, pitch, roll](https://en.wikipedia.org/wiki/Aircraft_principal_axes)) determinam a orientação do globo em relação à superfície sobre a qual ele está sendo projetado.*


### Uma Visita aos Tipos Específicos de Projeção

[**Projeções cilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical) mapeiam a esfera para um cilindro ao redor, depois desenrolam o cilindro. Se o eixo principal do cilindro estiver orientado no sentido norte-sul, os meridianos são mapeados para linhas retas. As projeções [pseudo-cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com outros meridianos "curvando-se" para longe do centro.


In [ ]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

- [Equiretangular](https://en.wikipedia.org/wiki/Equirectangular_projection) (`equirectangular`): Escala os valores das coordenadas `lat` e `lon` diretamente.
- [Mercator](https://en.wikipedia.org/wiki/Mercator_projection) (`mercator`): Projeta em um cilindro, usando `lon` diretamente, mas sujeitando `lat` a uma transformação não linear. Linhas retas preservam as direções constantes da bússola ([linhas de rumo](https://en.wikipedia.org/wiki/Rhumb_line)), tornando essa projeção bem adequada para navegação. No entanto, áreas no extremo norte ou sul podem ser grandemente distorcidas.
- [Mercator Transversal](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção mercator, mas com o cilindro de contorno rotacionado para um eixo transversal. Enquanto a projeção padrão de Mercator tem maior precisão ao longo do equador, a projeção Mercator Transversal é mais precisa ao longo do meridiano central.
- [Natural Earth](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudo-cilíndrica projetada para mostrar o mundo inteiro em uma única visão.
<br/><br/>


[**Projeções cônicas**](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera para um cone e depois desenrolam o cone sobre o plano. As projeções cônicas são configuradas por dois _paralelos padrão_, que determinam onde o cone intersecta a esfera.


In [ ]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

- [Área Cônica Igual](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): Projeção cônica que preserva a área. A forma e a distância não são preservadas, mas a precisão é razoável dentro dos paralelos padrão.
- [Cônica Equidistante](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): Projeção cônica que preserva a distância ao longo dos meridianos e paralelos padrão.
- [Cônica Conformal](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva a forma (ângulos locais), mas não a área ou a distância.
- [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção cônica igual área com paralelos padrão otimizados para criar mapas dos Estados Unidos.
- [Albers EUA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos Estados Unidos da América. Essa projeção costura três projeções Albers com parâmetros diferentes para os EUA continentais, Alasca e Havai.


[**Projeções azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeie a esfera diretamente no plano.

In [ ]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

- [Área Azimutal Igual](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta com precisão a área em todas as partes do globo, mas não preserva a forma (ângulos locais).
- [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Preserva a distância proporcional do centro da projeção para todos os outros pontos no globo.
- [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): Projeta um hemisfério visível em um plano distante. Aproximadamente corresponde a uma visão da Terra do espaço.
- [Estereográfica](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): Preserva a forma, mas não a área ou a distância.
- [Gnômica](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente em um plano tangente. [Círculos máximos](https://en.wikipedia.org/wiki/Great_circle) ao redor da Terra são projetados para linhas retas, mostrando o caminho mais curto entre os pontos.
